In [96]:
import openai
import pandas as pd
import numpy as np
import yaml
import os
from openai.embeddings_utils import cosine_similarity
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
import swifter
from configparser import ConfigParser

In [245]:
llm = ChatOpenAI(openai_api_key = openai_config['api_key'])
llm(temp)

AIMessage(content='Seafood', additional_kwargs={}, example=False)

## Prompt Template

#### Category

In [339]:
llm = ChatOpenAI(model = 'gpt-4', temperature= .7, openai_api_key = api_key )

level1_template = """
You are an expert in understanding Retail SKUs. Among different categories of snacks it is your job to find out
the particular category for the SKU given below.

SKU: {SKU}

Return only the output and nothing else. If you don't know the answer, reply Not Applicable.
Strictly review your output.
"""
level1_prompt_template = PromptTemplate(input_variables=['SKU'], template = level1_template)
level1_chain = LLMChain(llm=llm, prompt= level1_prompt_template, output_key="category")

level1_chain.run(SKU = 'NONGSHIM SHRIMP SNACK 75G')

'Seafood Snacks'

In [341]:
level2_template = '''

You are an expert in understanding Retails SKUs. 
Please find out the brand name or comapny name, which creates snacks of the category: {category} and has the sku: {SKU}

Return only the output and nothing else. If you don't know the answer, reply Not Applicable.
Strictly review your output.
'''

level2_prompt_template = PromptTemplate(input_variables=['SKU', 'category' ], template = level2_template)
level2_chain = LLMChain(llm=llm, prompt= level2_prompt_template, output_key="brand_name")

level2_chain.run(SKU = 'TONG GARDEN PEANUT & CRACKER 50G', category = 'peanuts')

'Tong Garden'

In [343]:
level3_template = '''

You are an expert in understanding Retails SKUs. 
What is the pack size for the SKU : {SKU}, given the category is {category} and brand name is {brand_name}

Use your best judgement with the category and brand name.
Your output should be between Small, Medium, Large, Not Applicable. Return only the output and nothing else. 
Strictly review your output.
'''

level3_prompt_template = PromptTemplate(input_variables=['SKU', 'category', 'brand_name' ], template = level3_template)
level3_chain = LLMChain(llm=llm, prompt= level3_prompt_template, output_key= "pack_size")

level3_chain.run(SKU = 'TONG GARDEN PEANUT & CRACKER 50G', category = 'peanuts', brand_name = 'Tong Garden')

'Small'

In [350]:
level4_template = '''

You are an expert in understanding Retails SKUs. 
For a given snack which belongs to category: {category} , made by this brand name: {brand_name}, available in pack size: {pack_size}.
What is the flavour or taste of snack with SKU: {SKU}?


Return only the output and nothing else. If you don't know the answer, reply Don't Know.
Strictly review your output.
'''

level4_prompt_template = PromptTemplate(input_variables=['SKU', 'category', 'brand_name', 'pack_size' ], template = level4_template)
level4_chain = LLMChain(llm=llm, prompt= level4_prompt_template, output_key= "flavour")

level4_chain.run(SKU = 'TONG GARDEN PEANUT & CRACKER 50G', category = 'peanuts', brand_name = 'Tong Garden', pack_size = 'Small' )

'Peanut & Cracker'

In [361]:
level5_template = '''
You are an expert in understanding Retails SKUs. 
What would be the type of snack for SKU: {SKU}.
Given the category of the snack is {category} , made by brand/company {brand_name} available in pack size {pack_size} and the flavour is {flavour}  and 

Return only the output and nothing else. If you don't know the answer, reply Don't Know.

Strictly review your output.
'''

level5_prompt_template = PromptTemplate(input_variables=['SKU', 'category', 'brand_name', 'pack_size', 'flavour' ], template = level5_template)
level5_chain = LLMChain(llm=llm, prompt= level5_prompt_template, output_key= "type")

level5_chain.run(SKU = 'TONG GARDEN PEANUT & CRACKER 50G', category = 'peanuts', brand_name = 'Tong Garden', pack_size = 'Small', flavour = 'Cracker' )

'Peanuts'

In [353]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[level1_chain, level2_chain, level3_chain, level4_chain, level5_chain],
    input_variables=["SKU"],
    output_variables=['category' ,"brand_name", "pack_size", "flavour", "type"],
    verbose=True)

overall_chain({ "SKU": "TONG GARDEN PEANUT & CRACKER 50G"})



> Entering new SequentialChain chain...

> Finished chain.


{'SKU': 'TONG GARDEN PEANUT & CRACKER 50G',
 'category': 'Nuts and Seeds',
 'brand_name': 'Tong Garden',
 'pack_size': 'Small',
 'flavour': 'Peanut & Cracker',
 'type': 'Nuts and Seeds'}

In [354]:
overall_chain({ "SKU": "PIATTOS SNACK SP 75G"})



> Entering new SequentialChain chain...

> Finished chain.


{'SKU': 'PIATTOS SNACK SP 75G',
 'category': 'Chips',
 'brand_name': 'Not Applicable',
 'pack_size': 'Small',
 'flavour': "Don't Know",
 'type': "Don't Know"}

In [355]:
input_dataset = pd.read_excel(r"C:\Users\66660\Documents\Projects\bcn-labs-string-parsing\data\raw\input_dataset.xlsx")
sku = input_dataset['Description'].str[13:].to_list()

top20_sku = sku[0:20]

c:\Users\66660\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [356]:
overall_chain({"SKU" : top20_sku[0]})



> Entering new SequentialChain chain...



> Finished chain.


{'SKU': 'KOREAN COLOR CURRY POPCORN 80G',
 'category': 'Popcorn',
 'brand_name': 'Not Applicable',
 'pack_size': 'Small',
 'flavour': 'Color Curry',
 'type': 'Korean Color Curry Popcorn'}

In [357]:
overall_chain({"SKU" : top20_sku[1]})



> Entering new SequentialChain chain...

> Finished chain.


{'SKU': 'TONG GARDEN SUNFLOWER SEEDS NO SHELL 130G',
 'category': 'Seeds & Nuts',
 'brand_name': 'Tong Garden',
 'pack_size': 'Small',
 'flavour': "Don't Know",
 'type': 'Sunflower Seeds'}

In [358]:
overall_chain({"SKU" : top20_sku[2]})



> Entering new SequentialChain chain...

> Finished chain.


{'SKU': 'WHITE PUMPKIN SEED 500G',
 'category': 'Seeds & Nuts',
 'brand_name': 'Not Applicable',
 'pack_size': 'Medium',
 'flavour': "Don't Know",
 'type': 'White Pumpkin Seed'}

In [359]:
overall_chain({"SKU" : "HWA YUAN POTATO CHIPS TAIWAN POPCORN CHICKEN FLA"})




> Entering new SequentialChain chain...

> Finished chain.


{'SKU': 'HWA YUAN POTATO CHIPS TAIWAN POPCORN CHICKEN FLA',
 'category': 'Potato Chips',
 'brand_name': 'Hwa Yuan',
 'pack_size': 'Not Applicable',
 'flavour': 'Popcorn Chicken',
 'type': 'Popcorn Chicken Flavored Potato Chips'}

In [364]:
for i in range(len(top20_sku)):
    print(overall_chain({"SKU" : top20_sku[i]}))



> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'KOREAN COLOR CURRY POPCORN 80G', 'category': 'Popcorn', 'brand_name': 'Not Applicable', 'pack_size': 'Small', 'flavour': 'Color Curry', 'type': 'Color Curry Popcorn'}


> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'TONG GARDEN SUNFLOWER SEEDS NO SHELL 130G', 'category': 'Seeds and Nuts', 'brand_name': 'Tong Garden', 'pack_size': 'Small', 'flavour': "Don't Know", 'type': 'Sunflower Seeds'}


> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'WHITE PUMPKIN SEED 500G', 'category': 'Seeds & Nuts', 'brand_name': 'Not Applicable', 'pack_size': 'Medium', 'flavour': "Don't Know", 'type': 'White Pumpkin Seed'}


> Entering new SequentialChain chain...

> Finished chain.
{'SKU': 'TAISUN TREATZ VEGGIE STICKS 250G', 'category': 'Veggie Snacks', 'brand_name': 'Taisun', 'pack_size': 'Medium', 'flavour': "Don't Know", 'type': 'Veggie Sticks'}


> Entering new SequentialChain chain...

> Finish